# Laboratorio 3: Programación Lineal Mixta (MA4702, otoño 2023)

Miércoles 17 de Mayo.<br/>
Profesor: Marcos Kiwi.<br/>
Auxiliares: Benjamín Jauregui y Cristián Palma.<br/>

## Preliminares

Este laboratorio consta de 3 etapas. Un trabajo introductorio, una etapa presencial y otra no-presencial.
La nota global del laboratorio será 30 % (TI) + 40 % (TP) + 30 % (TNP), donde (TI) es la nota del trabajo introductorio, (TP) la nota del trabajo presencial y (TNP) la nota del trabajo no presencial.

El trabajo introductorio es de carácter estrictamente individual. La etapa presencial y no-presencial son de carácter grupal. 


# Laboratorio 3: Trabajo No-presencial (TNP)


## Instrucciones.

En la parte no presencial del laboratorio, realice las siguientes actividades:

* Complete el ejercicio 3.
* Envíe este archivo .ipynb por el módulo de tareas de UCursos en la TAREA: Laboratorio 3 completo.
* Plazo de entrega: **Sábado 20 de abril a las 18:00** (se descontará 0.5 pts. por cada hora de atraso).


<div class="alert alert-block alert-warning">
    Advertencia: Si en el archivo que entregue alguna de las celdas arroja un mensaje de error al ejecutar o el código entra en un loop infinito y no retorna con ninguna salida, Ud. deberá agregar una nota al respecto, explicando cuál cree que es el problema. Si no lo hace, se le penalizará adicionalmente.               
</div>


## A. Datos del grupo

<div class="alert alert-block alert-success">

- Número o identificador del Grupo: 
    
- Integrante 1: XXX

- Integrante 2: YYY     </div>


## B. Preparación

Desempaque el `.zip` del laboratorio en su directorio de trabajo manteniendo la estructura de sub-directorios en que vienen empaquetados los archivos. 



## Ejercicio 3

En esta etapa continuamos desarrollando la temática discutida en la Etapa Presencial y consideramos la misma notación utilizada.

Para cada una de las instancias proveídas en la Etapa Presencial del Laboratorio, se le pide resolver iterativamente $(RMP)$ y el correspondiente $(DMP)$ incorporando en cada iteración una nueva columna a $(RMP)$ de acuerdo al resultado del *Pricing Problem* y hasta que el valor óptimo del *Pricing Problem* sea no-negativo.

Para realizar lo pedido, use como base el código de la siguiente celda:

In [ ]:
using DelimitedFiles, JuMP, Gurobi;           
dirInst = "dirTNP";            # Directorio con el archivo de instancias para el ejercicio 1.
const GUROBI_ENV = Gurobi.Env() 

function funcRMP(W,Paths,Pcst)

    rmp = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0, "Presolve" => 0)); 

    #
    # Agregar código a partir de aquí.
    #    
    n = size(W,1);
    N = size(Paths,1);
    @variable(rmp, x[1:N] >= 0);
    @constraint(rmp, cnstr[v in 2:n-1], sum( x[i] for i in 1:N if v in Paths[i] ) >= 1);
    @objective(rmp, Min, sum(Pcst[i]*x[i] for i in 1:N) );
    #
    # Parar de agregar código aquí.
    #
    
    return rmp, cnstr;  # cnstr[] debe ser un arreglo de referencias a las |C| restricciones (*) de (RMP).
                        # Ver: https://jump.dev/JuMP.jl/stable/manual/constraints/#jump_constraints
end

En la siguiente celda copie el código de la función `pricingProb(W,y,m)` que creó durante la Etapa Presencial (complete y verifique su código si no lo alcanzó a hacer durante la Etapa Presencial).

In [ ]:
function pricingProb(W,y,m)
    # W: matriz de n x n que codifica a \widetilde{w}.
    # y: vector de dimensión n de variables duales de (RMP) como en el enunciado (con y_s = y_t = 0).
    # m: largo máximo de los (s,t)-dicaminos.
    #
    
    ...
    
end

Complete el código de la siguiente celda (copie, donde se indica, el código que agregó en el Ejercicio 1b)) e inicialice (donde se índica) el arreglo con los valores de las variables duales. Finalmente, adapte (donde se indica) el código para resolver a optimalidad la relajación de $(MP)$ (hasta que el *Pricing Problem* asociado a $(RMP)$ se haga no-negativo).  

In [ ]:
tmpFrm=[]; tmpRes=[]; cardC=[]; val=[]; name=[]; sts=[];  
for fn in readdir(dirInst)
    W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')
    n = size(W,1);
    s = 1;
    t = n;
    Paths = [];  Pcst = [];

    #
    # Inicialización de \mathcal{Q}: Copiar código del Ejercicio 1b)
    #

    ...
    
    #
    # Parar de copiar código del Ejercicio 1 aquí.
    # 
    
    valPrcDicamn = -Inf;
    while valPrcDicamn < 0.0
        append!( name, [fn] );                                          # Guarda nombre del archivo con instancia
        append!( tmpFrm, @elapsed mdl,cnstr=funcRMP(W,Paths,Pcst) );    # Guarda tiempo requerido en formular (RMP)
        append!( tmpRes, @elapsed optimize!(mdl) );	                    # Guarda tiempo requerido en resolver (RMP)
        append!( cardC, size(W,1)-2 );                                  # Guarda cardinalidad de conjunto C
        if termination_status(mdl) == MOI.OPTIMAL                       # Guarda status de término y valor objetivo.
            append!( sts, ["optimo"] );
            append!( val, objective_value(mdl) );
        elseif termination_status(mdl) == MOI.INFEASIBLE
            append!( sts, ["infactible"] );
            append!( val, [Inf]);
        elseif termination_status(mdl) == MOI.DUAL_INFEASIBLE
            append!( sts, ["no_acotado"] );
            append!( val, [-Inf]);
        end       
    
  
        # Inicializar el arreglo y[] con los valores de las variables duales. Puede serle útil
        # consultar la documentación de las funciones has_duals() y dual() de JuMP has_duals().
        y = zeros(n)

        # Termino de inicialización de arreglo y[]
        
        dicamn, valPrcDicamn = pricingProb(W,y,3);

        # Comenzar a adaptar código aquí
        
        ...
        
        # Terminar de adaptar código aquí
    end
end

#
# NO MODIFICAR A CONTINUACION
#
# Imprime estadísticas relativas a las instancias resueltas.
println()
for i in 1:length(name)
    println( "Instancia = ", name[i] );
    println( "  |C| = ", cardC[i] );
    println( "  Tmp. formulación = ", tmpRes[i] );
    println( "  Tmp. resolución = ", tmpRes[i] );
    println( "  valor obj. = ", val[i] );
    println( "  status = ", sts[i] );
    println( "  prcDicamn = ", prcDicamn[i] );
    println( "  prcPesoDicamn = ", prcPesoDicamn[i] );
    println()
end


Observe que en algunos casos la convergencia es a un valor no integral, lo que significa que la solución óptima de la relajación de $(MP)$ encontrada no es integral -- para llegar a una solución integral habría que hacer *Branch&Price*.

<div class="alert alert-block alert-danger">Es aconsejable (pero no lo incluya en la respuesta que entregue) que genere resultados intermedios. En particular, que le permitan visualizar cómo va decreciendo el valor óptimo de $(RMP)$.</div>